In [1]:
"""(m1.py) Model of small shop with balking"""

from SimPy.Simulation import *
import random
import numpy
import math

## Useful extras ----------
def conf(L):
    """confidence interval"""
    lower = numpy.mean(L) - 1.96*numpy.std(L)/math.sqrt(len(L))
    upper = numpy.mean(L) + 1.96*numpy.std(L)/math.sqrt(len(L))
    return (lower,upper)

## Model ----------
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = random.expovariate(lamb)
            yield hold, self, t

class Arrival(Process):
    """an arrival"""

    def run(self):
        num_in_system = len(G.server.waitQ)+len(G.server.activeQ)
        if (num_in_system>=3):
            G.balkmon.observe(1)
        else:
            G.balkmon.observe(0)
            yield request, self, G.server
            if (random.random()<0.4):
                # child
                t = random.uniform(1,5)
            else:
                # adult
                t = random.uniform(2,8)
            yield hold, self, t
            yield release, self, G.server

class G:
    server = 'dummy'
    balkmon = 'Monitor'

def model(c, N, lamb, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c)
    G.balkmon = Monitor()
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb))
    simulate(until=maxtime)

    # gather performance measures
    Prop_balk = G.balkmon.mean()
    Rate_balk = G.balkmon.total()/now()
    return(Prop_balk,Rate_balk)

## Experiment ----------------
for R in range(10,21):
    allProp_balk = []
    allRate_balk = []
    for k in range(50):
        seed = 123*k
        result = model(c=1, N=10000, lamb=R/60.0,
                      maxtime=2000000, rvseed=seed)
        allProp_balk.append(result[0])
        allRate_balk.append(result[1])
    print "Arrival rate (per hour):",R
    print "    Estimate of prop balk:",numpy.mean(allProp_balk)
    print "    Conf int of prop balk:",conf(allProp_balk)
    print "    Estimate of rate balk:",numpy.mean(allRate_balk)
    print "    Conf int of rate balk:",conf(allRate_balk)


SyntaxError: invalid syntax (<ipython-input-1-781cb6e9817f>, line 75)